In [1]:
import pandas as pd 
import numpy as np 




# Pre Proccessing 

In [2]:
#import data 
cidc_df = pd.read_csv('/Users/Desktop/Network Detection /V1/cidc_all_merged_2.csv')

In [3]:
cidc_df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', ' Label', 'Init_Win_bytes_forward'],
      dtype='object')

In [4]:
#checking to see if any null values 

cidc_df.isnull().sum()

 Destination Port          0
 Flow Duration             0
 Total Fwd Packets         0
 Total Backward Packets    0
 Label                     0
Init_Win_bytes_forward     0
dtype: int64

In [5]:
#normalization 
from sklearn.preprocessing import MinMaxScaler

#copying orginal df
df_norm = cidc_df.copy()

# create a scaler object
scaler = MinMaxScaler()

#numerical columns 
num_cols = [' Flow Duration', ' Total Fwd Packets',' Total Backward Packets',  'Init_Win_bytes_forward']

# fit and transform the data
df_norm[num_cols] = scaler.fit_transform(cidc_df[num_cols])

df_norm

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Label,Init_Win_bytes_forward
0,22,1.491667e-06,0.000000,0.000003,BENIGN,0.004440
1,60148,7.999999e-07,0.000000,0.000007,BENIGN,0.003723
2,123,8.329999e-04,0.000000,0.000003,BENIGN,0.000000
3,123,3.085833e-04,0.000000,0.000003,BENIGN,0.000000
4,0,9.263445e-01,0.000664,0.000000,BENIGN,0.000000
...,...,...,...,...,...,...
2830738,55641,5.166666e-07,0.000000,0.000010,BENIGN,0.000015
2830739,45337,1.916666e-06,0.000005,0.000003,BENIGN,0.002106
2830740,22,1.156300e-02,0.000182,0.000158,BENIGN,0.445572
2830741,22,1.833333e-06,0.000000,0.000003,BENIGN,0.004440


In [6]:
#encoding Labels


from sklearn.preprocessing import LabelBinarizer

#copying df 
cidc_sklearn = cidc_df.copy()

#setting name 
lb = LabelBinarizer()

#encoding 
lb_results = lb.fit_transform(cidc_sklearn[' Label'])
lb_results_df  = pd.DataFrame(lb_results, columns=lb.classes_)
 

# Replace Categotical Data with Encoded Data
df_norm = df_norm.drop([' Label'],axis=1)
df_norm = pd.concat([df_norm, lb_results_df], axis=1)

#checking data 
print('Shape of dataframe:', df_norm.shape)
df_norm.head(5)

Shape of dataframe: (2830743, 20)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Init_Win_bytes_forward,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Heartbleed,Infiltration,PortScan,SSH-Patator,Web Attack ï¿½ Brute Force,Web Attack ï¿½ Sql Injection,Web Attack ï¿½ XSS
0,22,1.491667e-06,0.000000,0.000003,0.004440,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,60148,7.999999e-07,0.000000,0.000007,0.003723,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,123,8.329999e-04,0.000000,0.000003,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,123,3.085833e-04,0.000000,0.000003,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,9.263445e-01,0.000664,0.000000,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
df_norm.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Init_Win_bytes_forward', 'BENIGN', 'Bot',
       'DDoS', 'DoS GoldenEye', 'DoS Hulk', 'DoS Slowhttptest',
       'DoS slowloris', 'FTP-Patator', 'Heartbleed', 'Infiltration',
       'PortScan', 'SSH-Patator', 'Web Attack ï¿½ Brute Force',
       'Web Attack ï¿½ Sql Injection', 'Web Attack ï¿½ XSS'],
      dtype='object')

In [8]:
##splitting dataset 

# setting features 
feature = df_norm[[' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',  'Init_Win_bytes_forward']]


#setting target
target = df_norm.drop(columns=[' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',  'Init_Win_bytes_forward'])


# set training and testing data 
# 70% of data is used to train & 30% of the data is used to test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature , target, 
                                                    shuffle = True, 
                                                    test_size=0.3, 
                                                    random_state=42)





# Model 

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

clf = KNeighborsClassifier(n_neighbors=5, algorithm='auto')

clf.fit(X_train,y_train)

y_pred3 = clf.predict(X_test)

#accuracy 
acc3 =   accuracy_score(y_test,y_pred3)

In [10]:
print("Accuracy is: " , acc3)

Accuracy is:  0.9916829855055739
